<a href="https://colab.research.google.com/github/Y-Yahyaoui/GNSS/blob/main/Infrastructure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install georinex

In [ ]:
import georinex as gr
from math import sin, atan, sqrt, cos, pi, isnan
from datetime import datetime
import numpy as np

In [ ]:
def extract_params(file_name, sat_name, epoch):
    nav = gr.load(file_name)
    M0 = nav.sel(sv=sat_name)['M0'].values[epoch]
    e = nav.sel(sv=sat_name)['Eccentricity'].values[epoch]
    sqrtA = nav.sel(sv=sat_name)['sqrtA'].values[epoch]
    Omega0 = nav.sel(sv=sat_name)['Omega0'].values[epoch]
    Io = nav.sel(sv=sat_name)['Io'].values[epoch]
    omega = nav.sel(sv=sat_name)['omega'].values[epoch]
    DeltaN = nav.sel(sv=sat_name)['DeltaN'].values[epoch]
    cuc = nav.sel(sv=sat_name)['Cuc'].values[epoch]
    cus = nav.sel(sv=sat_name)['Cus'].values[epoch]
    crc = nav.sel(sv=sat_name)['Crc'].values[epoch]
    crs = nav.sel(sv=sat_name)['Crs'].values[epoch]
    cis = nav.sel(sv=sat_name)['Cis'].values[epoch]
    cic = nav.sel(sv=sat_name)['Cic'].values[epoch]
    OmegaDot = nav.sel(sv=sat_name)['OmegaDot'].values[epoch]
    IDOT = nav.sel(sv=sat_name)['IDOT'].values[epoch]
    toe = nav.sel(sv=sat_name)['Toe'].values[epoch]
    t_h = float(str(nav.sel(sv='G08').coords['time'][epoch].values).split('T')[1].split(':')[0])*3600
    t_min = float(str(nav.sel(sv='G08').coords['time'][epoch].values).split('T')[1].split(':')[1])*60
    t_s = float(str(nav.sel(sv='G08').coords['time'][epoch].values).split('T')[1].split(':')[2])
    t = t_h + t_min + t_s
    datetimestr = str(nav.sel(sv=sat_name).coords['time'].values[epoch]).split('T')[0]
    datetime_object = datetime.strptime(datetimestr, '%Y-%m-%d')
    T_sat = datetime_object.isoweekday()*24*3600 + t
    return M0, e, sqrtA, Omega0, Io, omega, DeltaN, cuc, cus, crc, crs, cis, cic, OmegaDot, IDOT, toe, t, T_sat

In [ ]:
def satellite_position(file_name, sat_name, epoch):
    #Extraction des paramètes
    #--------------------------------------------------
    M0, e, sqrtA, Omega0, Io, omega, DeltaN, cuc, cus, crc, crs, cis, cic, OmegaDot, IDOT, toe, t, T_sat = extract_params(file_name, sat_name, epoch)
    #--------------------------------------------------
    #Calculs
    t_k = t-toe
    u = sqrt(3.986004418*10**14)
    if t_k>302400 :
        t_k = t_k - 604800
    elif t_k<-302400 :
        t_k = t_k + 604800
    #Caclul de l'anomalie moyenne pour t_k
    M_k = M0 + ((u/sqrtA**3)+DeltaN)*t_k
    #Calcul de l'anomalie excentrique
    E0 = M_k
    E_k = E0 + e*sin(E0)
    while abs(E0-E_k)<0.000001 :
        E0 = E_k
        E_k = M_k + e*sin(E0)
    omega_E = 7.292115e-5
    #Calcul de l'anomalie vraie
    v_k = atan((sqrt(1-e**2)*sin(E_k))/(cos(E_k)-e))
    #Calcul de l'arg de la latitude Uk
    u_k = omega + v_k + cuc*cos(2*(omega+v_k)) + cus*sin(2*(omega+v_k))
    #Calcul de la distance radiale rk
    r_k = (sqrtA**2)*(1-e*cos(E_k)) + crc*cos(2*(omega+v_k)) + crs*sin(2*(omega+v_k))
    #Calcul de l'inclinaison ik du plan orbital
    i_k = Io + IDOT*t_k + cic*cos(2*(omega+v_k)) + cis*sin(2*(omega+v_k))
    #Calcul de la longitude du noeud ascendant lambda_k
    lambda_k = Omega0 + (OmegaDot - omega_E)*t_k - omega_E*toe
    #Matrices de rotation
    R3lambda = np.array([[cos(-lambda_k), -sin(-lambda_k), 0], [sin(-lambda_k), cos(-lambda_k), 0], [0,0,1]])
    R3uk = np.array([[cos(-u_k), -sin(-u_k), 0], [sin(-u_k), cos(-u_k), 0], [0,0,1]])
    R1ik = np.array([[1,0,0], [0, cos(-i_k), -sin(-i_k)], [0, sin(-i_k), cos(-i_k)]])
    #Calcul des coordonnées dans le référentiel CTS
    dotproduct = np.dot(np.dot(R3lambda, R1ik), R3uk)
    vector = np.array([[r_k], [0], [0]])
    coords = np.dot(dotproduct, vector)
    return coords, T_sat

In [ ]:
def app_coords(X, Y, Z, D):
    # X Y Z D sont respectivement des tableaux de 4 coordonnées et pseudo-distances des 4 premiers satellites, leur but est de calculer les coordonnées approximatives de la station 
    B=np.array([[X[0]-X[1],Y[0]-Y[1],Z[0]-Z[1]],[X[1]-X[2],Y[1]-Y[2],Z[1]-Z[2]],[X[2]-X[3],Y[2]-Y[3],Z[2]-Z[3]]])
    C=np.array([[D[1]**2-D[0]**2-(X[1]**2+Y[1]**2+Z[1]**2)+(X[0]**2+Y[0]**2+Z[0]**2)],[D[2]**2-D[1]**2-(X[2]**2+Y[2]**2+Z[2]**2)+(X[1]**2+Y[1]**2+Z[1]**2)],[D[3]**2-D[2]**2-(X[3]**2+Y[3]**2+Z[3]**2)+(X[2]**2+Y[2]**2+Z[2]**2)]])*0.5
    X0=np.dot(np.linalg.inv(B),C)
    return X0


def jacobienne(X, Y, Z, D, X0):
    # X Y Z D sont respectivement des tableaux de toutes les coordonnées et pseudo-distances de tous les satellites, X0 sont les coordonnées approchés de la station
    A=np.zeros((len(X), 3), dtype=float)
    W=np.zeros((len(X), 1), dtype=float)
    for i in range(len(D)):
        E=sqrt((X0[0][0]-X[i])**2+(X0[1][0]-Y[i])**2+(X0[2][0]-Z[i])**2)
        A[i][0]=(X0[0][0]-X[i])/E
        A[i][1]=(X0[1][0]-Y[i])/E
        A[i][2]=(X0[2][0]-Z[i])/E
        W[i][0]=E-D[i]
    return A, W

def CMC(A, W, X0):
    N=np.dot(A.transpose(),A)
    U=np.dot(A.transpose(),W)
    X=np.dot(np.linalg.inv(N),U)
    Xcorr=X0-X
    return Xcorr

In [ ]:
nav = gr.load('/content/vald3640.21o')
nav1 = gr.load('/content/vald3640.21n')


satellites = nav1.coords['sv'].values
satellitesO = nav.coords['sv'].values
values_epoch1 = np.zeros((len(satellites),18))
satellites_in_range_epoch1 = []
satellites_in_range_epoch2 = []


values_epoch2 = np.zeros((len(satellites),18))
for i in range(len(satellites)):
    values_epoch1[i] = np.array([extract_params('vald3640.21n', satellites[i], 0)])
    values_epoch2[i] = np.array([extract_params('vald3640.21n', satellites[i], 1)])
    if len(values_epoch1[i][~np.isnan(values_epoch1[i])]) > 2:
        satellites_in_range_epoch1.append(satellites[i])
    if len(values_epoch2[i][~np.isnan(values_epoch2[i])]) > 2:
        satellites_in_range_epoch2.append(satellites[i])

coords_epoch1 = {}
coords_epoch2 = {}
T_sat_epoch1 = {}
T_sat_epoch2 = {}

for i in range(len(satellites_in_range_epoch1)):
    coords_epoch1[satellites_in_range_epoch1[i]], T_sat_epoch1[satellites_in_range_epoch1[i]] = satellite_position('vald3640.21n',satellites_in_range_epoch1[i],0) 
for i in range(len(satellites_in_range_epoch2)):
    coords_epoch2[satellites_in_range_epoch2[i]], T_sat_epoch2[satellites_in_range_epoch2[i]] = satellite_position('vald3640.21n',satellites_in_range_epoch2[i],1) 


for i in range(len(coords_epoch1)):
    for j in range(len(coords_epoch2)):
        if not list(coords_epoch2)[j] in list(coords_epoch1):
            coords_epoch1[list(coords_epoch2)[j]] = coords_epoch2[list(coords_epoch2)[j]]
            T_sat_epoch1[list(T_sat_epoch2)[j]] = T_sat_epoch2[list(T_sat_epoch2)[j]]


dict={}


c = 3*10**8
for i in range(119):
    for j in range(len(satellitesO)):
        if (not isnan(nav.sel(sv=satellitesO[j])["P2"].values[i])) and (satellitesO[j] in satellites_in_range_epoch1 or satellitesO[j] in satellites_in_range_epoch2) and not satellitesO[j] in dict:
            dict[satellitesO[j]] = nav.sel(sv=satellitesO[j])["P2"].values[0]

X=[]
Y=[]
Z=[]
D=[]
for i in range(len(dict)):
    X.append(coords_epoch1[list(dict)[i]][0])
    Y.append(coords_epoch1[list(dict)[i]][1])
    Z.append(coords_epoch1[list(dict)[i]][2])
    D.append(dict[list(dict)[i]])


X = np.array(X)
Y = np.array(Y)
Z = np.array(Z)
D = np.array(D)

for i in range(len(D)):
    if isnan(D[i]):
        D = np.delete(D, i)
        X = np.delete(X, i)
        Y = np.delete(Y, i)
        Z = np.delete(Z, i)


X0 = app_coords(X, Y, Z, D)
A, W = jacobienne(X, Y, Z, D, X0)
X_corr = CMC(A, W, X0)
print(coords_epoch1)
print(X_corr)

{'G03': array([[  3961014.3459775],
       [-17575451.5240497],
       [ 19653949.1937669]]), 'G29': array([[  3583844.73436282],
       [-25144165.32021204],
       [  7632414.22853904]]), 'G31': array([[-23978500.39067741],
       [ -4410192.41532748],
       [ 11208576.55082116]]), 'G01': array([[13855279.72547076],
       [21515103.09788518],
       [ 6127730.20188543]]), 'G04': array([[  7423330.05233627],
       [ 14731354.42935557],
       [-20799209.42435418]]), 'G07': array([[ 3632125.29855373],
       [24597114.051065  ],
       [ 8980341.98994046]]), 'G08': array([[-15437974.46430942],
       [ -4167976.33937957],
       [-21244531.35918032]]), 'G09': array([[  3540146.82881312],
       [-21703496.52688536],
       [ 14946717.89784869]]), 'G10': array([[-12667946.4103483 ],
       [ 12226456.48010979],
       [-20108960.31028885]]), 'G13': array([[-17745570.93312223],
       [  4740397.39830386],
       [ 19087572.09014963]]), 'G14': array([[ 11671476.64537003],
       [-131